In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import sys;
sys.path.append(r'C:\Monitor\\')
sys.path.append(r'\\ac-geneva-24\E\grains trading\Streamlit\Monitor\\')

from datetime import datetime as dt

import numpy as np
import pandas as pd

import Utilities.Modeling as um
import Utilities.Weather as uw
import Utilities.GLOBAL as GV
import Utilities.Utilities as uu

import Models.Soybean_USA_Yield_GA as sy

import warnings # supress warnings
warnings.filterwarnings('ignore')
pd.options.mode.chained_assignment = None

<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [4]:
f='GA_soy'
dm_best = {}
dm_best[f] = uu.deserialize(f)
r=dm_best[f]

r.keys()

Start Deserializing: GA_soy
Done Deserializing: GA_soy


dict_keys(['best_fitness', 'model', 'MAE', 'MAPE', 'fitness', 'corr', 'cv_p', 'cv_r_sq', 'cv_MAE', 'cv_MAPE', 'cv_corr'])

In [ ]:
print(len(r['model']))
print(len(r['corr']))

print(r['corr'])